In [1]:
import numpy as np
import Utilityfunctions as utils
import jax.numpy as jnp
import ssr_kronvec_jax as ssr
import ssr_likelihood_jax as jax_lik
import ssr_likelihood as pyt

In [2]:
n = 3
theta = utils.random_theta(n, 0.0)
n_dat = 100
dat = np.random.binomial(n=1, p=0.6, size=n_dat*(2*n+1))
dat = jnp.array(dat).reshape((n_dat, -1))
dat = dat.at[:,-1].set(1)
dat = jnp.column_stack((dat, jnp.ones(n_dat, dtype=int)))
lam1 = 0.5
lam2 = 1.5

In [ ]:
dat_point = dat.at[0,0:2*n+1].get()
m = dat_point.sum()
p0 = np.zeros(2**m)
p0[0] = 1
pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)

In [ ]:
dat_point = jnp.array([1,1,0,0,1,1,1], dtype = int)
m = dat_point.sum()
p0 = np.zeros(2**m)
p0[0] = 1
pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = dat_point, transpose = False)
count = dat_point[1:2*n:2].sum()
latent_dist = jnp.zeros(2**(count))
print(utils.ssr_obs_dist(pTh1, dat_point, n, obs_prim=True))
inds = ssr.obs_inds(pTh1, dat_point, latent_dist, obs_prim=True)
utils.trunk_states(dat_point)[2**(m-1)+inds[0].at[0].get()]

In [ ]:
jax_lik.log_prob_coupled(dat, theta, lam1, lam2)

In [ ]:
jax_lik.log_prob_single(dat, jnp.array(theta), lam1)

In [ ]:
jax_lik.grad_single(dat, jnp.array(theta), lam1)

In [7]:
state = jnp.ones(7, dtype=int)#dat.at[2, 0:2*n+1].get()
m = state.sum()
count = state.at[0:2*n+1:2].get().sum()
latent_state = state.at[0:2*n:2].get()#jnp.append(state.at[1:2*n:2].get(), 1) 
print(state)
print(latent_state)
latent_dist = jnp.zeros(2**(count))
p0 = jnp.zeros(2**int(state.sum()))
p0 = p0.at[0].set(1.)
pTh1 = jax_lik.R_i_inv_vec(log_theta = theta, x = p0, lam = lam1,  state = state, transpose = False)

#jax_lik._grad_coupled(theta, lam1, lam2, state, p0, latent_dist, latent_state, True)
inds = ssr.obs_inds(pTh1, state, latent_dist, obs_prim=False)
pTh1_obs = pTh1.at[inds].get()
nk = pTh1_obs.sum()
q = jnp.zeros_like(p0)
print(utils.trunk_states(state)[inds])
print(pTh1_obs)
pos = 2**(m-1) + inds.at[0].get()
q = q.at[pos].set(1/nk)
#print(q)
print(utils.trunk_states(state)[pos])
# Actual gradient
q = ssr.marg_transp(q, state, marg_met=False, marg_seeding=False)
print(q)
q = jax_lik.R_i_inv_vec(theta, q, lam1, state, transpose = True)
g_1 = jax_lik.x_partial_Q_y(theta, q, pTh1, state)
g_1

[1 1 1 1 1 1 1]
[1 1 1]
['0101010' '1101010' '0111010' '1111010' '0101110' '1101110' '0111110'
 '1111110' '0101011' '1101011' '0111011' '1111011' '0101111' '1101111'
 '0111111' '1111111']
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.78057840e-02
 6.75252086e-05 7.78667090e-05 2.11256520e-05 2.29074758e-05
 2.96817650e-03 2.62657195e-01 1.62437104e-03 1.23637825e-01]
0101011
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


Array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [ ]:
theta